Importações

In [ ]:
import tensorflow
from tensorflow import keras
import imutils
from imutils import video
import cv2
import time
import numpy as np

Criando função que detecta a máscara

In [ ]:
def detector(frame, face, mask):
    (altura, largura) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224), (104.0, 177.0, 123.0))

    face.setInput(blob)
    deteccoes = face.forward()
    print(deteccoes.shape)

    faces = []
    locs = []
    preds = []

    for indice in range(0, deteccoes.shape[2]):
        confianca = deteccoes[0, 0, indice, 2]
        if confianca > 0.5:
            box = deteccoes[0, 0, indice, 3:7] * np.array([largura, altura, largura, altura])
            (startX, startY, endX, endY) = box.astype("int")


            (startX, startX) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(largura - 1, endX), min(altura - 1, endY))

            face = frame[startX:endY, startY:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = keras.preprocessing.image.img_to_array(face)
            face = keras.applications.mobilenet_v2.preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = mask.predict(faces, batch_size=32)
    
    return (locs, preds)

Carregando o modelo de detecção de rosto

In [ ]:
prototxtPath = r"deploy.prototxt"
weightsPath = r"res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

Carregando o modelo que foi criado para detectar máscara

In [ ]:
maskNet = keras.models.load_model("modelo_reconhecimento_mascara.h5")

Inicializando o vídeo

In [ ]:
vs = video.VideoStream(src=0).start()

Loop dos frames do vídeo

In [ ]:
while True:
	frame = vs.read()
	frame = imutils.resize(frame, width=400)
	
	(locs, preds) = detector(frame, faceNet, maskNet)

	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# intrrupção com o 'q'
	if key == ord("q"):
		break

Limpando e parando o stream do vídeo

In [ ]:
cv2.destroyAllWindows()
vs.stop()